In [19]:
from ipynb.fs.full.LocationCleaningScript import clean_location
from ipynb.fs.full.cleanIndustries import cleanIndustries
#from ipynb.fs.full.CurrencyConverter import *
#from ipynb.fs.full.NormalizeWorkExperience import NormalizeExperience
import pandas 
import urllib.request
import json
from datascience import *
def RemoveSpacesFromTitles(df):
    
    df.columns = [c.replace(' ', '_') for c in df.columns]
    return df
def NormalizeExperience(filename):
    df = pandas.read_csv(filename)
    df = df.replace(to_replace = ['5-7 years'], value = '5 - 7 years')
    df = df.replace(to_replace = ['1 year or less'], value = '0 - 1 years')
    return df
  

#x = x.to_df()

In [15]:
class currencyConverter:
    rates = {}
    def __init__(self, url):
        data = urllib.request.urlopen(url).read().decode('utf-8')
        f = json.loads(data)
#         print(f)
        self.rates = f["rates"]
        
        
    def convert(self, amount, f, t):
        output = 0
        if f != 'USD':
            output = amount / self.rates[f] * self.rates['USD']
            return output
        return amount
        
def convertCurrency(df):
    
    for i in range(len(df['Currency'])):
        f = df['Currency'][i]
        df['Annual_Salary'][i] = df['Annual_Salary'][i].replace(',', '')
        salary = df['Annual_Salary'][i]
        otherComp = df['Other_Compensation'][i]
        if f == 'Other':
            otherCur = df['Other'][i]
            if otherCur in c.rates:
                df['Annual_Salary'][i] = c.convert(float(salary), otherCur, 'USD')
                if otherComp != "" and otherComp != "0":
                    df['Other_Compensation'][i] = c.convert(float(otherComp), otherCur, 'USD')
        elif f != 'USD':
            if f == 'AUD/NZD':
                f = 'AUD'
            df['Annual_Salary'][i] = c.convert(float(salary), f, 'USD')
            df['Other_Compensation'][i] = c.convert(float(otherComp), f, 'USD')
        if not pd.isna(df['Other_Compensation'][i]):
            df['Annual_Salary'][i] = "{:.2f}".format(float(df['Other_Compensation'][i]) + float(df['Annual_Salary'][i]))
    return df

c = currencyConverter("http://data.fixer.io/api/latest?access_key=238f1925907e0f01a7fa1dc2b4042967")

In [25]:
table = NormalizeExperience('../2021_Results.csv')

table = RemoveSpacesFromTitles(table)

table = clean_location(Table.from_df(table))

table = table.to_df()
table = convertCurrency(table)
table
#table = cleanIndustries(table)
#table
#table.to_csv("CleanedData2021.csv")

<ipython-input-15-ebb7c733c0ba>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = df['Annual_Salary'][i].replace(',', '')
<ipython-input-15-ebb7c733c0ba>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = "{:.2f}".format(float(df['Other_Compensation'][i]) + float(df['Annual_Salary'][i]))
<ipython-input-15-ebb7c733c0ba>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary']

,Timestamp,Age,Industry,Job_Title,Annual_Salary,Currency,Work_Experience,Context_for_Job,Other,Other_Compensation,Context,Country,State,City,Highest_Level_of_Education,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,55000.00,USD,5 - 7 years,nan,nan,0.000000,nan,United States of America,Massachusetts,Boston,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,82958.78,GBP,8 - 10 years,nan,nan,5662.715592,nan,United States of America,,Cambridge,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,34000,USD,2 - 4 years,nan,nan,NaN,nan,United States of America,Tennessee,Chattanooga,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,65000.00,USD,8 - 10 years,nan,nan,3000.000000,nan,United States of America,Wisconsin,Milwaukee,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,67000.00,USD,8 - 10 years,nan,nan,7000.000000,nan,United States of America,South Carolina,Greenville,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26334,5/25/2021 10:39:03,45-54,Computing or Tech,Information Technology Professional,76480,USD,31 - 40 years,nan,nan,NaN,nan,United States of America,Minnesota,Minneapolis,College degree,Woman,White
26335,5/25/2021 10:47:06,45-54,Health care,System Analyst,75000.00,USD,11 - 20 years,nan,nan,0.000000,nan,United States of America,Texas,Austin,Master's degree,Woman,White
26336,5/25/2021 10:50:28,25-34,Computing or Tech,Database Software Engineer,93000,USD,11 - 20 years,nan,nan,NaN,nan,United States of America,Oregon,Portland,Master's degree,Woman,White
26337,5/25/2021 10:55:57,25-34,Business or Consulting,Senior Associate,161000.00,USD,8 - 10 years,nan,nan,10000.000000,nan,United States of America,New York,New York,Master's degree,Man,Asian or Asian American
